eva  - compiler for the reals

nominal completion nd slotted egraphs
f(e2(a), e3(a,b)) -> e1(a,b)

op = 1+e) + d
d for denormla

posits

x*x + y*y

datflow - fatser
daisy
fluctuat

optimzation - optimization
fptaylor
precisa

non modular, no loops
SAS modulao

hugo tool.

<https://github.com/soarlab/FPTaylor>
<https://github.com/malyzajko/daisy>

(Handbook of mathemtical functions)
Kalmand filters

heat equation loop

satire tool and fluctat
<https://github.com/arnabd88/Satire>

<https://github.com/soarlab/gelpia>

conditionals. are still tough

a and b are *atoms*
So we are 3 flat instead of 2 flat

Propel talk diseaqlity forbid. Good chat with him.

https://github.com/metalibm/metalibm

https://gappa.gitlabpages.inria.fr/gappa/index.html

In [2]:
%%file /tmp/testgappa.g
{ x in [-2,2] -> x * x in ? }

Writing /tmp/testgappa.g


In [6]:
! gappa -Blatex /tmp/testgappa.g | pandoc -f latex -t markdown

Results:
  x * x in [0, 4]
Under the following hypotheses $$\label{g1}
\left( \left| \mathit{x} \right| \le2 \land \neg\left( \mathit{x} \times \mathit{x} \in [0,4] \right) \right),$$
one can deduce the following properties: $$\label{l2}
\neg\left( \mathit{x} \times \mathit{x} \in [0,4] \right)$$ by using
([\[g1\]](#g1){reference-type="ref" reference="g1"}), and selecting a
component. $$\label{l5}
\left| \mathit{x} \right| \le2$$ by using
([\[g1\]](#g1){reference-type="ref" reference="g1"}), and selecting a
component. $$\label{l4}
\left| \mathit{x} \right| \le 2$$ by using
([\[l5\]](#l5){reference-type="ref" reference="l5"}), and theorem
`abs_of_bnd_o`. $$\label{l3}
\mathit{x} \times \mathit{x} \in [0,4]$$ by using
([\[l4\]](#l4){reference-type="ref" reference="l4"}), and theorem
`square`. $$\label{l1}
\bot$$ by using ([\[l2\]](#l2){reference-type="ref" reference="l2"}),
([\[l3\]](#l3){reference-type="ref" reference="l3"}), and discarding
contradictory literals.


In [ ]:
import subprocess
def gappa(f):
    


In [ ]:
ex1 = """{ x in [0,1] -> float<ieee_32,ne>(x * float<ieee_32,ne>(1 - x)) in ? }"""
gappa(ex1)
